# Task 01 Supervised Learning Model

## 1. Data Visualization

In [24]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [25]:
items = pd.read_csv('datasets/item.csv')
sales = pd.read_csv('datasets/sales.csv')
supermarkets = pd.read_csv('datasets/supermarkets.csv')
promotions = pd.read_csv('datasets/promotion.csv')

In [26]:
items.head()
# sales.head()
# supermarkets.head()
# promotions.head()

,code,descrption,type,brand,size
0,3000005040,AUNT JEM ORIGINAL PANCAKE MIX,Type 1,Aunt Jemima,2 LB
1,3000005070,A/JEM COMPLETE PANCAKE MI,Type 1,Aunt Jemima,32 OZ
2,3000005300,AJ BUTTERMILK PANCAKE MIX,Type 1,Aunt Jemima,32 OZ
3,3000005350,A J BTRMLK COMP PNCK MIX,Type 1,Aunt Jemima,1 LB
4,1600015760,BC PANCAKE MIX BUTTERMILK,Type 1,Bisquick,6.75 OZ


In [27]:
def summarize_column_types(df):
    dtype_mapping = {
        'int64': 'integer',
        'float64': 'float',
        'object': 'string',
        'bool': 'boolean',
        'datetime64[ns]': 'datetime',
        'category': 'categorical',
    }
    
    #generate the summary
    summary = df.dtypes.reset_index()
    summary.columns = ['Column Name', 'Pandas Data Type']
    summary['Primitive Datatype'] = summary['Pandas Data Type'].astype(str).fillna('unknown')
    return summary

In [28]:
summary_items = summarize_column_types(items)
summary_supermarkets = summarize_column_types(supermarkets)
summary_promotions = summarize_column_types(promotions)
summary_sales = summarize_column_types(sales)

print(f'{summary_supermarkets} \n')
print(f'{summary_sales} \n')
print(f'{summary_supermarkets} \n')
print(f"{summary_items}\n")

      Column Name Pandas Data Type Primitive Datatype
0  supermarket_No            int64              int64
1     postal-code            int64              int64 

    Column Name Pandas Data Type Primitive Datatype
0          code            int64              int64
1        amount          float64            float64
2         units            int64              int64
3          time            int64              int64
4      province            int64              int64
5          week            int64              int64
6    customerId            int64              int64
7   supermarket            int64              int64
8        basket            int64              int64
9           day            int64              int64
10      voucher            int64              int64 

      Column Name Pandas Data Type Primitive Datatype
0  supermarket_No            int64              int64
1     postal-code            int64              int64 

  Column Name Pandas Data Type Primitive Datat

### Finding Duplicates

In [29]:
duplicate_supermarkets = supermarkets[supermarkets.duplicated(subset=['supermarket_No', 'postal-code'])]
print(f'Number of duplicate records found: {len(duplicate_supermarkets)}')

duplicate_items = supermarkets[supermarkets.duplicated()]
print(f'Number of duplicated Items found: {len(duplicate_items)}')

duplicate_sales = sales[sales.duplicated()]
print(f"Number of Duplicated Sales : {len(duplicate_sales)}")

duplicated_promotions = promotions[promotions.duplicated()]
print(f'Number of Duplicated Promotions: {len(duplicated_promotions)}')

Number of duplicate records found: 0
Number of duplicated Items found: 0
Number of Duplicated Sales : 0
Number of Duplicated Promotions: 0


### Finding Features with Missing Values

In [30]:
def display_missing_values(df, dataset_name):
    
    df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    print(f"\nMissing value information for {dataset_name}")
    
    #count missnig values in each column 
    missing_info = df.isnull().sum()
    missing_info = missing_info[missing_info > 0] #filter columns with atleast 1 missing value
    
    if not missing_info.empty:
        print('Columns with Missing Values: ')
        print(missing_info)
        print(f"\nTotal columns with missing values: {len(missing_info)}")
    
    else:
        print("No missing values in the dataset")
    

In [31]:
display_missing_values(items, 'Items')
display_missing_values(sales, 'Sales')
display_missing_values(supermarkets, 'supermarkets')
display_missing_values(promotions, 'promotions')


Missing value information for Items
Columns with Missing Values: 
size    3
dtype: int64

Total columns with missing values: 1

Missing value information for Sales
No missing values in the dataset

Missing value information for supermarkets
No missing values in the dataset

Missing value information for promotions
No missing values in the dataset


`Based on the information and the relationship between datasets, the ideal problem to solve would be to see the efficiency of the promotion in supermarkets.`
`Due to the inconsistencies in size attribute in items. It will be omitted. Even though it would vastly impact in sales forecasting and efficiency power, inconsistencies in the values can reduce the performance of the model`

### Cleaning Data Inconsistencies 

1. Removing Incosnsitent values by converting to the right data format.
2. Standardizing the strings 
3. Feature Engineering 

### Renaming columns for dataframe merging

In [54]:
promotions.rename(columns={'supermarkets':'supermarket'}, inplace=True)
supermarkets.rename(columns={'supermarket_No':'supermarket'}, inplace=True)

In [55]:
supermarkets.head()

,supermarket,postal-code
0,199,30319
1,200,30134
2,201,30066
3,202,31093
4,203,30542


### Handling Missing Values (Discarding sizes due to inconsistencies)

In [56]:
items = items.drop(columns=['size'], errors='ignore')
items.shape

(927, 4)

In [57]:
#merging sales and items 
sales_items = pd.merge(sales, items, on='code', how='left')

In [58]:
sales_items.head()

,code,amount,units,time,province,week,customerId,supermarket,basket,day,voucher,descrption,type,brand
0,7680850106,0.80,1,1100,2,1,125434,244,1,1,0,BARILLA ANGEL HAIR,Type 2,Barilla
1,3620000470,3.59,1,1100,2,1,125434,244,1,1,0,BERTOLLI TOM&BASIL SAUCE,Type 3,Bertolli
2,1800028064,2.25,1,1137,2,1,108320,244,2,1,0,H J PANCK BTRMLK COMP MIX,Type 1,Hungry Jack
3,9999985067,0.85,1,1148,2,1,162016,244,3,1,0,PRIVATE LABEL VERMICELLI,Type 2,Private Label
4,9999985131,2.19,1,1323,2,1,89437,244,4,1,0,PRIVATE LABEL IMPORTED LASAGNA,Type 2,Private Label Premium


#### Verifying if Sales and Sales Items are equal in row size

In [59]:
if sales.shape[0] == sales_items.shape[0]:
    print("Sales Shape", sales.shape[0])
    print("Sales_items Shape", sales_items.shape[0])

Sales Shape 1048575
Sales_items Shape 1048575


In [60]:
promotions.head()

,code,supermarket,week,feature,display,province
0,2700042240,285,91,Not on Feature,Mid-Aisle End Cap,2
1,2700042292,285,92,Interior Page Feature,Not on Display,2
2,2700042274,285,92,Interior Page Feature,Not on Display,2
3,2700042273,285,92,Interior Page Feature,Not on Display,2
4,2700042254,285,92,Interior Page Feature,Not on Display,2


In [61]:
sales_items_promotions = pd.merge(sales_items, promotions, on=['code', 'supermarket', 'province'], how='left')

In [63]:
sales_items_promotions.head()

,code,amount,units,time,province,week_x,customerId,supermarket,basket,day,voucher,descrption,type,brand,week_y,feature,display
0,7680850106,0.8,1,1100,2,1,125434,244,1,1,0,BARILLA ANGEL HAIR,Type 2,Barilla,83.0,Interior Page Feature,Not on Display
1,7680850106,0.8,1,1100,2,1,125434,244,1,1,0,BARILLA ANGEL HAIR,Type 2,Barilla,72.0,Interior Page Feature,Not on Display
2,7680850106,0.8,1,1100,2,1,125434,244,1,1,0,BARILLA ANGEL HAIR,Type 2,Barilla,68.0,Interior Page Feature,Not on Display
3,7680850106,0.8,1,1100,2,1,125434,244,1,1,0,BARILLA ANGEL HAIR,Type 2,Barilla,67.0,Interior Page Feature,Not on Display
4,7680850106,0.8,1,1100,2,1,125434,244,1,1,0,BARILLA ANGEL HAIR,Type 2,Barilla,46.0,Interior Page Feature,Not on Display


In [64]:
final_dataset = pd.merge(
    sales_items_promotions, 
    supermarkets,
    on='supermarket',
    how='left'
)

In [65]:
final_dataset.shape

(4184347, 18)

In [66]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4184347 entries, 0 to 4184346
Data columns (total 18 columns):
 #   Column       Dtype  
---  ------       -----  
 0   code         int64  
 1   amount       float64
 2   units        int64  
 3   time         int64  
 4   province     int64  
 5   week_x       int64  
 6   customerId   int64  
 7   supermarket  int64  
 8   basket       int64  
 9   day          int64  
 10  voucher      int64  
 11  descrption   object 
 12  type         object 
 13  brand        object 
 14  week_y       float64
 15  feature      object 
 16  display      object 
 17  postal-code  int64  
dtypes: float64(2), int64(11), object(5)
memory usage: 574.6+ MB


In [67]:
final_dataset.to_csv('final_dataset', index=False)

In [68]:
# Remove duplicates from the final dataset
final_dataset_cleaned = final_dataset.drop_duplicates()

# Display the number of rows before and after removing duplicates
print(f"Number of rows before removing duplicates: {len(final_dataset)}")
print(f"Number of rows after removing duplicates: {len(final_dataset_cleaned)}")

# # Save the cleaned dataset
# final_dataset_cleaned.to_csv('final_sales_dataset_cleaned.csv', index=False)
# print("\nCleaned dataset (duplicates removed) saved as 'final_sales_dataset_cleaned.csv'.")


Number of rows before removing duplicates: 4184347
Number of rows after removing duplicates: 4184347


In [69]:
final_dataset_cleaned = final_dataset_cleaned.dropna()

In [70]:
final_dataset_cleaned.shape

(3844549, 18)